In [1]:
import pandas as pd
import sklearn


In [2]:
#!pip install pyarrow

In [3]:
jan_23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
feb_23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [4]:
feb_23

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.40,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.00,-1.00,-0.5,0.00,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.00,1.00,0.5,0.00,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.90,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.00,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913950,2,2023-02-28 23:46:00,2023-03-01 00:05:00,NaN,4.65,NaN,None,249,140,0,20.22,0.00,0.5,4.84,0.0,1.0,29.06,NaN,NaN
2913951,2,2023-02-28 23:26:02,2023-02-28 23:37:10,NaN,2.47,NaN,None,186,79,0,13.66,0.00,0.5,2.65,0.0,1.0,20.31,NaN,NaN
2913952,2,2023-02-28 23:24:00,2023-02-28 23:38:00,NaN,3.49,NaN,None,158,143,0,17.64,0.00,0.5,0.00,0.0,1.0,21.64,NaN,NaN
2913953,2,2023-02-28 23:03:00,2023-02-28 23:10:00,NaN,2.13,NaN,None,79,162,0,13.56,0.00,0.5,2.63,0.0,1.0,20.19,NaN,NaN


In [5]:
jan_23["duration"] = jan_23["tpep_dropoff_datetime"] - jan_23["tpep_pickup_datetime"]

In [6]:
jan_23.duration.std()

Timedelta('0 days 00:42:35.661074517')

In [7]:
jan_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype          
---  ------                 -----          
 0   VendorID               int64          
 1   tpep_pickup_datetime   datetime64[ns] 
 2   tpep_dropoff_datetime  datetime64[ns] 
 3   passenger_count        float64        
 4   trip_distance          float64        
 5   RatecodeID             float64        
 6   store_and_fwd_flag     object         
 7   PULocationID           int64          
 8   DOLocationID           int64          
 9   payment_type           int64          
 10  fare_amount            float64        
 11  extra                  float64        
 12  mta_tax                float64        
 13  tip_amount             float64        
 14  tolls_amount           float64        
 15  improvement_surcharge  float64        
 16  total_amount           float64        
 17  congestion_surcharge   float64        
 18  ai

In [8]:
minutes = jan_23["duration"].dt.total_seconds() / 60
jan_23_treated = jan_23[(minutes >= 1) & (minutes <= 60)]


minutes_treated = jan_23_treated["duration"].dt.total_seconds() / 60


In [9]:
jan_23_treated.shape[0] / jan_23.shape[0] 

0.9812202822125979

In [10]:
# turn the dataframe into a list of dictionaries

df_jan23 = jan_23_treated[["PULocationID","DOLocationID"]].astype("str")

dic_jan23 = df_jan23.to_dict(orient='records')
dic_jan23

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'},
 {'PULocationID': '234', 'DOLocationID': '68'},
 {'PULocationID': '79', 'DOLocationID': '264'},
 {'PULocationID': '164', 'DOLocationID': '143'},
 {'PULocationID': '138', 'DOLocationID': '33'},
 {'PULocationID': '33', 'DOLocationID': '61'},
 {'PULocationID': '79', 'DOLocationID': '186'},
 {'PULocationID': '90', 'DOLocationID': '48'},
 {'PULocationID': '113', 'DOLocationID': '255'},
 {'PULocationID': '237', 'DOLocationID': '239'},
 {'PULocationID': '143', 'DOLocationID': '229'},
 {'PULocationID': '137', 'DOLocat

In [11]:
# fit a dict vectorizer
from sklearn.feature_extraction import DictVectorizer

vect = DictVectorizer()
x = vect.fit_transform(dic_jan23)

In [12]:
# Train a plain linear regression
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(x,minutes_treated)

LinearRegression()

In [13]:
# calculate rmse of model on training data

from sklearn.metrics import mean_squared_error

(mean_squared_error(minutes_treated,lr.predict(x)))**(1/2)


7.649261027919939

In [14]:
# evaluating the model




In [15]:

# calculate duration

feb_23["duration"] = feb_23["tpep_dropoff_datetime"] - feb_23["tpep_pickup_datetime"]

#calculate minutes
minutes_feb23 = feb_23["duration"].dt.total_seconds() / 60

# Eliminate outliers
feb_23_treated = feb_23[(minutes_feb23 >= 1) & (minutes_feb23 <= 60)]

minutes_feb23_treated = feb_23_treated["duration"].dt.total_seconds() / 60


# generate the dict

dict_feb23 = feb_23_treated[["PULocationID","DOLocationID"]].astype("str").to_dict(orient='records')

x_feb23 = vect.transform(dict_feb23)

(mean_squared_error(minutes_feb23_treated,lr.predict(x_feb23)))**(1/2)


7.811832638273232